In [3]:
import pandas as pd
import numpy as np
from dstapi import DstApi

In [28]:
ind = DstApi('FOLK1A')

In [32]:
tabsum = ind.tablesummary()

for variable in tabsum['variable name']:
    print(variable+':')
    display(ind.variable_levels(variable))

Table FOLK1A: Folketal den 1. i kvartalet efter område, køn, alder, civilstand og tid
Last update: 2023-02-10T08:00:00
OMRÅDE:


,id,text
0,000,Hele landet
1,084,Region Hovedstaden
2,101,København
3,147,Frederiksberg
4,155,Dragør
...,...,...
100,773,Morsø
101,840,Rebild
102,787,Thisted
103,820,Vesthimmerlands


KØN:


,id,text
0,TOT,I alt
1,1,Mænd
2,2,Kvinder


ALDER:


,id,text
0,IALT,I alt
1,0,0 år
2,1,1 år
3,2,2 år
4,3,3 år
...,...,...
122,121,121 år
123,122,122 år
124,123,123 år
125,124,124 år


CIVILSTAND:


,id,text
0,TOT,I alt
1,U,Ugift
2,G,Gift/separeret
3,E,Enke/enkemand
4,F,Fraskilt


Tid:


,id,text
0,2008K1,2008K1
1,2008K2,2008K2
2,2008K3,2008K3
3,2008K4,2008K4
4,2009K1,2009K1
...,...,...
56,2022K1,2022K1
57,2022K2,2022K2
58,2022K3,2022K3
59,2022K4,2022K4


In [63]:
kom = []
for row, value in zip(ind.variable_levels('OMRÅDE')['id'],ind.variable_levels('OMRÅDE')['text']):
    if 'Region' not in value:
        kom.append(row)

In [66]:
params = {'table':'FOLK1A',
         'format':"BULK",
         'variables':[{'code':'OMRÅDE','values':kom},
                     {'code':'KØN','values':['*']},
                     {'code':'ALDER','values':['*']},
                     {'code':'Tid','values':['*']},
                     {'code':'CIVILSTAND','values':['TOT']}]}
data= ind.get_data(params)

In [99]:
data = pd.read_csv('FOLK1A',index_col=[0])
data['TID'] = pd.to_datetime(data.reset_index()['TID'].apply(change_format))
data.set_index('TID',inplace=True)
#data.resample('Y').last()
data = data[data['KØN']=='I alt']

In [131]:
data_re = data[['OMRÅDE','ALDER','INDHOLD']].groupby(['OMRÅDE','ALDER']).last()
data_re = data_re.reset_index().set_index('OMRÅDE')
data_re['ALDER'] = data_re['ALDER'].str.strip(' år')
data_re


SyntaxError: unmatched ']' (3266771268.py, line 3)

In [90]:
#data.set_index('TID',inplace=True)
#d = datetime.datetime.strptime('2011-06-09', '%YYYY%QQ')
def change_format(inputs):
    inputs = inputs.replace('K','Q')
    return inputs



In [91]:
data

,OMRÅDE,KØN,ALDER,TID,CIVILSTAND,INDHOLD
0,Haderslev,Mænd,55 år,2016-04-01,I alt,382
1,Haderslev,Mænd,56 år,2016-04-01,I alt,410
2,Haderslev,Mænd,57 år,2016-04-01,I alt,393
3,Haderslev,Mænd,58 år,2016-04-01,I alt,410
4,Haderslev,Mænd,59 år,2016-04-01,I alt,366
...,...,...,...,...,...,...
1878821,Egedal,Kvinder,22 år,2022-01-01,I alt,112
1878822,Egedal,Kvinder,23 år,2022-01-01,I alt,101
1878823,Egedal,Kvinder,24 år,2022-01-01,I alt,90
1878824,Egedal,Kvinder,25 år,2022-01-01,I alt,102


In [ ]:
data.index = pd.to_datetime(data.index)

In [ ]:
def plot_graph(year):
    # Filter the DataFrame based on the selected year
    filtered_df = df[df['Year'] == year]

    # Create the bar plot
    plt.bar(filtered_df['Category'], filtered_df['Value'])
    plt.title(f"Values for Year {year}")
    plt.xlabel("Category")
    plt.ylabel("Value")
    plt.show()

# Create the year slider
year_slider = widgets.IntSlider(
    value=df['Year'].min(),
    min=df['Year'].min(),
    max=df['Year'].max(),
    step=1,
    description="Year:",
    continuous_update=False,
)

# Create an interactive widget to update the graph based on the year slider
widgets.interactive(plot_graph, year=year_slider)




Obtaining file:///Users/frederikfindsen/Documents/%C3%98konomi/10.%20Semester/Introduction%20to%20Numerical%20Program/dstapi
  Running setup.py develop for dstapi
Note: you may need to restart the kernel to use updated packages.


In [1]:
from dstapi import DstApi